# B마트 인스타그램 데이터 분석을 통한 Q커머스 트렌드 조사 

* 관심 키워드: 편의점, 편의점배달, 편의점신상
* 편의점은 가장 자주, 간편하게 들리는 소매점이자, 다양한 신상품이 가득한 재밌는곳
* 소비자들의 관심사가 어떻게 변화하는지, 비슷해보이는 편의점들 사이에서 소비자들의 인식은 어떻게 다른지 분석하여 제품 기획이나 시장전략을 세워보고자 함.  

**그러나..**  

* 인스타 '#편의점'태그 검색 시 78만건 이상조회되어 크롤링이 어려울 것으로 판단
* 인스타 '#편의점신상' 검색 시 9.2만건 조회, organic아닌 광고인 경우가 많음

* 편의점에서는 16년도부터 배달 서비스를 시작 했고, 배달의 민족이 B마트 (지역별로 물류창고를 배치하고 자사 라이더를 통해 초소량 생필품을 30분~1시간 안에 배달 해주는 서비스)를 19년 말 출시한 뒤 1년만에 기존 편의점 배달시장의 50%를 장악
* B마트 검색시 게시물 5000여개로 크롤링에 적절해 보임
* 마켓컬리/쿠팡/SSG등 이커머스 기업과도 유사한 면이 있으나, 배송에 하루 이상이 걸리므로, 오프라인 편의점 배달서비스와 같이 퀵커머스로 분류 가능하다고 판단

* 관련기사: [배달의민족 ‘B마트’ 개시후 편의점 배달매출 반토막](https://www.donga.com/news/Economy/article/all/20201007/103296268/1)
* 관련기사: [논란의 'B마트', '배달의민족'보다 더 무섭게 성장했다](https://n.news.naver.com/article/293/0000034105)  
  
* B마트의 어떤점이 소비자들을 hooking했고, 다른 경쟁업체가 보완해야할 부분은 무엇일지 보고자 함

In [4]:
import requests
from bs4 import BeautifulSoup as bs
# import pymysql
from  selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time as time
import getpass
import urllib.request
import random
 
 
from time import sleep
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
driver = webdriver.Chrome("C:/Users/lhj90/chromedriver_win32/chromedriver.exe")
driver.get("https://www.instagram.com/accounts/login/")
driver.maximize_window()

In [11]:
username = getpass.getpass("Input ID : ")# User ID
password = getpass.getpass("Input PWD : ")# User PWD, 특히 getpass를 통해서 비밀번호 정보를 숨길 수도 있다. 잘 배워두자 
hashTag = input("Input HashTag # : ")# Search #
 
element_id = driver.find_element_by_name("username")
element_id.send_keys(username)
element_password = driver.find_element_by_name("password")
element_password.send_keys(password)
 
sleep(1.5)


Input ID : ········
Input PWD : ········
Input HashTag # : #b마트


In [12]:
##로그인버튼 클릭
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()

In [13]:
### 정보저장 나중에 하기 클릭
driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()

In [14]:
### 알림설정 나중에 하기 클릭
driver.find_element_by_css_selector('.aOOlW.HoLwm').click()

In [26]:
# instagram 태그검색 url패턴:  www.instagram.com/explore/tags/"검색하고자 하는 패턴"
# url 함수만들기

def instatag_searching(tag):
    url= "https://www.instagram.com/explore/tags/" + tag
    return url

In [27]:
# 검색 단어 설정
tag = "b마트"  #유사단어: 비마트

In [28]:
# 검색하기
url = instatag_searching(tag)
driver.get(url)

In [29]:
##데이터를 저장할 Dictionary
insta_dict = {'id':[],
              'date': [],
              'like': [],
              'text': [],
              'hashtag':[]}

In [30]:
##첫 번째 게시물 클릭 
first_post = driver.find_element_by_class_name('eLAPa')
first_post.click()

In [31]:
seq = 0
start = time.time()
 
while True:
    try:
        if driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow'):
            if seq % 20 == 0:
                print('{}번째 수집 중'.format(seq), time.time() - start, sep = '\t')
 
 
 
            ## id 정보 수집
            try:
                info_id = driver.find_element_by_css_selector('h2._6lAjh').text
                insta_dict['id'].append(info_id)
            except:
                info_id = driver.find_element_by_css_selector('div.C4VMK').text.split()[0]
                insta_dict['id'].append(info_id)
 
 
            ## 시간정보 수집 
            time_raw = driver.find_element_by_css_selector('time.FH9sR.Nzb55')
            time_info = pd.to_datetime(time_raw.get_attribute('datetime')).normalize()
            insta_dict['date'].append(time_info)
 
            ## like 정보 수집
            try:
                driver.find_element_by_css_selector('button.sqdOP.yWX7d._8A5w5')
                like = driver.find_element_by_css_selector('button.sqdOP.yWX7d._8A5w5').text
                insta_dict['like'].append(like)
 
            except:
                insta_dict['like'].append('영상')
 
 
 
            ##text 정보수집
            raw_info = driver.find_element_by_css_selector('div.C4VMK').text.split()
            text = []
            for i in range(len(raw_info)):
                ## 첫번째 text는 아이디니까 제외 
                if i == 0:
                    pass
                ## 두번째부터 시작 
                else:
                    if '#' in raw_info[i]:
                        pass
                    else:
                        text.append(raw_info[i])
            clean_text = ' '.join(text)
            insta_dict['text'].append(clean_text)
 
            ##hashtag 수집
            raw_tags = driver.find_elements_by_css_selector('a.xil3i')
            hash_tag = []
            for i in range(len(raw_tags)):
                if raw_tags[i].text == '':
                    pass
                else:
                    hash_tag.append(raw_tags[i].text)
 
            insta_dict['hashtag'].append(hash_tag)
 
            seq += 1
    
            # 10개만 일단 수집해보자
            if seq == 10:
                break
 
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(1.5)
            
 
        else:
            break
            
    except:
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

0번째 수집 중	0.05409407615661621


In [32]:
# Data Frame형태로 보기
test = pd.DataFrame.from_dict(insta_dict)
test

,id,date,like,text,hashtag
0,skinnypig.creamery,2020-07-22 00:00:00+00:00,영상,"이제 스피가 먹고 싶을 때, B마트에서 한 개만 주문해도 지금 나우 바로 즉시 도착...","[#B마트, #이벤트, #태그이벤트, #개이득, #개이득, #참여완료, #개이득]"
1,eyesmag,2020-08-24 00:00:00+00:00,영상,삼양(@samyangfoods)과 배달의 민족(@baemin_official)이 손...,"[#eyesmag, #samyang, #buldak, #baemin, #bmart,..."
2,blogger_nanal,2020-12-16 00:00:00+00:00,영상,@romandyou 🥘 저 떡볶이 안 시켰는데요 🤣 와...,"[#제품제공, #롬앤, #롬앤X배민, #립볶ᄋ..."
3,convee4,2020-10-10 00:00:00+00:00,영상,대만에서 우유 튀김 드셔보신 분 손!!! 저는 발 들겠습니다. 이유는 모르겠지만 대...,"[#편선생_직접먹어봄, #편선생, #B마트, #비마트, #사옹원, #우유튀김, #사..."
4,sweetbalance_kr,2020-06-16 00:00:00+00:00,영상,:: 초소량 번쩍배달 ⚡🛵 B마트 입점! 드디어...,"[#스윗밸런스, #B마트, #하나만사도, #바로추..."
5,panhonesta,2020-12-21 00:00:00+00:00,영상,< X > 여러분! 빵어니스타 타르트가 B마트에 이...,"[#B마트, #빵어니스타]"
6,panhonesta,2021-03-08 00:00:00+00:00,영상,< X > 안녕하세요 여러분🙋🏻‍♀️ 월요일부터 기쁜 소식을 전해드리러 왔어요 :)...,"[#B마트, #빵어니스타]"
7,eats_better,2020-11-02 00:00:00+00:00,영상,이제 2종! 과 를 주문하면 바로 오는 B마트ᄋ...,"[#잇츠베러뉴스, #배달의민족, #B마트, #잇ᄎ..."
8,po___ong,2020-12-16 00:00:00+00:00,영상,여러분들의 성원에 힘입어 포옹이 배민 B ᄆ...,"[#배민, #B마트, #생식선생, #테리셔스]"
9,dear.yiseo,2020-09-02 00:00:00+00:00,영상,200902 . 2시부터 진짜 진짜 맛있는거 먹고...,"[#이서_탄수, #이서_먹빵, #이서_다앤하, #쑤..."
